# Stock Market Data

In [55]:
import pandas as pd
from datetime import datetime, timedelta, date
import csv


## Loadind the data

In [2]:
citi = pd.read_csv("C-intra.csv")
goldman = pd.read_csv("GS-intra.csv")
morgan = pd.read_csv("MS-intra.csv")

---

## Overview of the dataframe

In [3]:
citi.head()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,20191008,100000,67.050,67.05,66.34,66.34,113224
1,20191008,110000,66.320,66.50,66.03,66.50,163117
2,20191008,120000,66.495,66.84,66.19,66.65,105768
3,20191008,130000,66.650,67.17,66.60,66.91,91339
4,20191008,140000,66.920,67.03,66.87,66.87,100957


In [4]:
goldman.head()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,20191008,100000,197.59,197.71,196.28,196.30,22894
1,20191008,110000,196.30,197.50,195.94,197.50,28411
2,20191008,120000,197.46,198.30,196.61,197.39,25527
3,20191008,130000,197.41,198.78,197.21,197.98,20972
4,20191008,140000,198.01,198.56,197.86,198.05,10451


In [5]:
morgan.head()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,20191008,110000,39.93,40.025,39.78,40.01,33348
1,20191008,120000,40.00,40.150,39.75,40.00,34901
2,20191008,130000,39.96,40.150,39.92,40.02,21383
3,20191008,140000,39.99,40.190,39.99,40.08,13972
4,20191008,150000,40.10,40.150,39.94,39.98,29330


---

## Standardization of data

In [6]:
files = [citi , goldman, morgan]

#### Remove '<' and '>' from columns names

In [7]:
def columns_names(files):
    for data in files:
        data.columns = data.columns.str.replace('<', '').str.replace('>', '')
    

In [8]:
columns_names(files)

In [9]:
citi.head()

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL
0,20191008,100000,67.050,67.05,66.34,66.34,113224
1,20191008,110000,66.320,66.50,66.03,66.50,163117
2,20191008,120000,66.495,66.84,66.19,66.65,105768
3,20191008,130000,66.650,67.17,66.60,66.91,91339
4,20191008,140000,66.920,67.03,66.87,66.87,100957


#### Standadize date and time

In [10]:
def to_date(files):
    for data in files:
        data['DATE'] = data['DATE'].map(str) + data['TIME'].map(str)
        data["DATE"]= pd.to_datetime(data["DATE"],format="%Y%m%d%H%M%S")
        data.drop(['TIME'],axis=1,inplace=True)

In [11]:
to_date(files)

In [12]:
citi.head()

,DATE,OPEN,HIGH,LOW,CLOSE,VOL
0,2019-10-08 10:00:00,67.050,67.05,66.34,66.34,113224
1,2019-10-08 11:00:00,66.320,66.50,66.03,66.50,163117
2,2019-10-08 12:00:00,66.495,66.84,66.19,66.65,105768
3,2019-10-08 13:00:00,66.650,67.17,66.60,66.91,91339
4,2019-10-08 14:00:00,66.920,67.03,66.87,66.87,100957


---

## Add stock change for future correlation

The percentage change in stock market is calculated from the price the market had closed the hour before. In the case of the start of the day, we take the closing price from the previous day.

In [13]:
def add_change(files):
    for data in files:
        list_change = []
        for index, row in data.iterrows():
            if index == 0:
                list_change.append(0)
            else:
                res = ((row['CLOSE']) - (data['CLOSE'][index-1])) / (data['CLOSE'][index-1])
                list_change.append(res*100)
        data['%CHANGE'] = list_change

In [14]:
add_change(files)

In [15]:
citi.head()

,DATE,OPEN,HIGH,LOW,CLOSE,VOL,%CHANGE
0,2019-10-08 10:00:00,67.050,67.05,66.34,66.34,113224,0.000000
1,2019-10-08 11:00:00,66.320,66.50,66.03,66.50,163117,0.241182
2,2019-10-08 12:00:00,66.495,66.84,66.19,66.65,105768,0.225564
3,2019-10-08 13:00:00,66.650,67.17,66.60,66.91,91339,0.390098
4,2019-10-08 14:00:00,66.920,67.03,66.87,66.87,100957,-0.059782


---

####  %Change 1h after every day opening

In [16]:
def add_change2(files):
    for data in files:
        if (data.equals(morgan)):
            list_change = []
            for index, row in data.iterrows():
                if ((row['DATE'].to_pydatetime().hour == 12) & (index != 1)):
                    res = ((row['CLOSE']) - (data['CLOSE'][index - 2])) / (data['CLOSE'][index - 2])
                    list_change.append(res*100)
                else:
                    list_change.append(0)
                
        else:   
            list_change = []
            for index, row in data.iterrows():
                if ((row['DATE'].to_pydatetime().hour == 11) & (index != 1)):
                    res = ((row['CLOSE']) - (data['CLOSE'][index - 2])) / (data['CLOSE'][index - 2])
                    list_change.append(res*100)
                else:
                    list_change.append(0)
                
        data['%CHANGE_1H'] = list_change

In [17]:
add_change2(files)

In [21]:
citi.loc[:10]

,DATE,OPEN,HIGH,LOW,CLOSE,VOL,%CHANGE,%CHANGE_1H
0,2019-10-08 10:00:00,67.050,67.05,66.34,66.340,113224,0.000000,0.000000
1,2019-10-08 11:00:00,66.320,66.50,66.03,66.500,163117,0.241182,0.000000
2,2019-10-08 12:00:00,66.495,66.84,66.19,66.650,105768,0.225564,0.000000
3,2019-10-08 13:00:00,66.650,67.17,66.60,66.910,91339,0.390098,0.000000
4,2019-10-08 14:00:00,66.920,67.03,66.87,66.870,100957,-0.059782,0.000000
5,2019-10-08 15:00:00,66.890,67.11,66.63,66.775,103199,-0.142067,0.000000
6,2019-10-08 16:00:00,66.790,66.97,66.32,66.425,166816,-0.524148,0.000000
7,2019-10-09 10:00:00,66.900,67.18,66.79,67.035,50634,0.918329,0.000000
8,2019-10-09 11:00:00,67.030,67.36,66.85,67.120,86032,0.126799,1.046293
9,2019-10-09 12:00:00,67.160,67.49,66.97,67.290,53317,0.253278,0.000000


In [23]:
print(goldman)

                  DATE    OPEN    HIGH     LOW   CLOSE    VOL   %CHANGE  \
0  2019-10-08 10:00:00  197.59  197.71  196.28  196.30  22894  0.000000   
1  2019-10-08 11:00:00  196.30  197.50  195.94  197.50  28411  0.611309   
2  2019-10-08 12:00:00  197.46  198.30  196.61  197.39  25527 -0.055696   
3  2019-10-08 13:00:00  197.41  198.78  197.21  197.98  20972  0.298901   
4  2019-10-08 14:00:00  198.01  198.56  197.86  198.05  10451  0.035357   
5  2019-10-08 15:00:00  198.04  198.93  197.83  197.97  31249 -0.040394   
6  2019-10-08 16:00:00  198.29  198.36  197.11  197.51  34862 -0.232358   
7  2019-10-09 10:00:00  198.35  198.72  197.32  198.09  11351  0.293656   
8  2019-10-09 11:00:00  198.09  198.68  197.74  198.16  20769  0.035337   
9  2019-10-09 12:00:00  198.17  198.40  197.41  198.05  18331 -0.055511   
10 2019-10-09 13:00:00  198.12  198.47  197.67  197.97  19491 -0.040394   
11 2019-10-09 14:00:00  198.07  198.22  197.41  197.78  18849 -0.095974   
12 2019-10-09 15:00:00  1

In [52]:
# little hacky code to get changes of a day on one row and make the file titles same as the tweet files

temp = {}

for company in ["Citigroup_", "goldmansachs_", "morganstanley_"]:
    if company == "Citigroup_":
        comp = citi
    elif company == "goldmansachs_":
        comp = goldman
    else:
        comp = morgan
    day_old = ""
    i = 0
    for index, row in comp.iterrows():
        date = str(row["DATE"]).split(" ")
        day = date[0]
        time = date[1]

        # new day, make new key
        if day != day_old:
            day_old = day
            temp[company + day] = [row["%CHANGE"]]
        else:
            temp[company + day].append(row["%CHANGE"])
        i += 1
    
print(temp)

{'Citigroup_2019-10-08': [0.0, 0.24118179077479135, 0.22556390977444465, 0.3900975243810816, -0.05978179644297123, -0.14206669657544319, -0.5241482590790093], 'Citigroup_2019-10-09': [0.9183289424162581, 0.12679943313195788, 0.2532777115613851, 0.12631891811560964, 0.007421150278286386, 0.2077767883645007, -0.1481042654028352], 'Citigroup_2019-10-10': [2.4918421833283784, 0.17366136034732932, -0.05778676683038434, -0.4047412546978912, -0.26124818577649755, -0.11641443538998589, 0.029137529137523342], 'Citigroup_2019-10-11': [2.8109525196621132, -0.09916418756198808, 0.02127056154282554, -0.19848302261288803, 0.277008310249318, 0.07083156254426569, -0.8069082672706787], 'Citigroup_2019-10-14': [-0.2854288568574121, 0.08587376556462326, 0.6721006721006704, 0.07102272727272323, 0.01419446415896509, -0.25546409310245904, -0.08537279453614438], 'Citigroup_2019-10-15': [-0.17089148390772507, 2.482168330955791, -0.30623608017817217, 0.27925160569671686, -0.3480924533556113, 0.1117786782171475

In [57]:
# # write output to match output of prediction

with open('change.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',')
        filewriter.writerow(["File", "10", "11", "12", "13", "14", "15", "16"])
        for d in temp:
            change = temp[d]
            filewriter.writerow([d, temp[d][0], temp[d][1], temp[d][2], temp[d][3], temp[d][4], temp[d][5]])
